In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/dacon-dobe2/sample_submission.csv
/kaggle/input/dacon-dobe2/test/641.png
/kaggle/input/dacon-dobe2/train/가구수정/11.png
/kaggle/input/dacon-dobe2/train/터짐/94.png
/kaggle/input/dacon-dobe2/train/들뜸/48.png
/kaggle/input/dacon-dobe2/train/피스/48.png
/kaggle/input/dacon-dobe2/train/창틀,문틀수정/11.png
/kaggle/input/dacon-dobe2/train/녹오염/11.png
/kaggle/input/dacon-dobe2/train/석고수정/48.png
/kaggle/input/dacon-dobe2/train/훼손/1231.png
/kaggle/input/dacon-dobe2/train/꼬임/173.png
/kaggle/input/dacon-dobe2/train/울음/11.png
/kaggle/input/dacon-dobe2/train/걸레받이수정/173.png
/kaggle/input/dacon-dobe2/train/틈새과다/4.png
/kaggle/input/dacon-dobe2/train/오염/173.png
/kaggle/input/dacon-dobe2/train/곰팡이/94.png
/kaggle/input/dacon-dobe2/train/이음부불량/11.png
/kaggle/input/dacon-dobe2/train/면불량/94.png
/kaggle/input/dacon-dobe2/train/반점/1.png
/kaggle/input/dacon-dobe2/train/오타공/94.png
/kaggle/input/dacon-dobe2/train/몰딩수정/94.png
/kaggle/working/__notebook_source__.ipynb


## 01. 필요한 라이브러리 불러오기

In [ ]:
from PIL import Image
import cv2
import albumentations as A
from tqdm import tqdm
import glob

## 02. Data 불러오기

In [2]:
train = pd.DataFrame({'path':glob.glob('/kaggle/input/dacon-dobe2/train/*/*')})
test = pd.read_csv('/kaggle/input/dacon-dobe2/test.csv')

In [3]:
train['label'] = train['path'].apply(lambda x : x.split('/')[-2])
train

,path,label
0,/kaggle/input/dacon-dobe2/train/가구수정/11.png,가구수정
1,/kaggle/input/dacon-dobe2/train/가구수정/4.png,가구수정
2,/kaggle/input/dacon-dobe2/train/가구수정/9.png,가구수정
3,/kaggle/input/dacon-dobe2/train/가구수정/1.png,가구수정
4,/kaggle/input/dacon-dobe2/train/가구수정/2.png,가구수정
...,...,...
3452,/kaggle/input/dacon-dobe2/train/몰딩수정/19.png,몰딩수정
3453,/kaggle/input/dacon-dobe2/train/몰딩수정/110.png,몰딩수정
3454,/kaggle/input/dacon-dobe2/train/몰딩수정/26.png,몰딩수정
3455,/kaggle/input/dacon-dobe2/train/몰딩수정/79.png,몰딩수정


In [4]:
test['img_path'] = test['img_path'].apply(lambda x : x[1:])
test['img_path'] = '/kaggle/input/dacon-dobe2' + test['img_path']
test

,id,img_path
0,TEST_000,/kaggle/input/dacon-dobe2/test/000.png
1,TEST_001,/kaggle/input/dacon-dobe2/test/001.png
2,TEST_002,/kaggle/input/dacon-dobe2/test/002.png
3,TEST_003,/kaggle/input/dacon-dobe2/test/003.png
4,TEST_004,/kaggle/input/dacon-dobe2/test/004.png
...,...,...
787,TEST_787,/kaggle/input/dacon-dobe2/test/787.png
788,TEST_788,/kaggle/input/dacon-dobe2/test/788.png
789,TEST_789,/kaggle/input/dacon-dobe2/test/789.png
790,TEST_790,/kaggle/input/dacon-dobe2/test/790.png


## 03. Image Augmentation
### Class 불균형 및 Class가 너무 적은 것들이 존재하기 때문에 Augmentation으로 데이터 추가
- 석고수정, 들뜸, 피스 &rarr; 2배
- 창틀,문틀수정, 울음, 이음부불량, 녹오염 가구수정 &rarr; 3배
- 틈새과다, 반점 &rarr; 5배
- size : (300,300)

In [6]:
two_list = ["석고수정", "들뜸" ,'피스']
os.makedirs('train_300_2', exist_ok=True)

for category in two_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(2):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(), 
                             A.RandomBrightnessContrast(brightness_limit=0.2), A.Resize(300,300)]) # A.VerticalFlip()
            image_aug = aug(image=image)['image']
            cv2.imwrite('train_300_2/' + str(i) + category + name, image_aug)  ### augmentation 2번 시행할 때 'i'로 구분하지 않으면 덮어씌어짐
    
    
three_list = ["창틀,문틀수정","울음","이음부불량","녹오염",'가구수정']
os.makedirs('train_300_3', exist_ok=True)

for category in three_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(3):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(),
                             A.RandomBrightnessContrast(brightness_limit=0.2), A.Resize(300,300)])
            image_aug = aug(image=image)['image']
            cv2.imwrite('train_300_3/' + str(i) + category + name, image_aug)
    
    
five_list = ["틈새과다",'반점']
os.makedirs('train_300_5', exist_ok=True)

for category in five_list:
    for path in tqdm(train[train['label']==category]['path']):
        for i in range(6):
            name = path.split('/')[-1]
            image = cv2.imread(path)
            aug = A.Compose([A.VerticalFlip(),A.Rotate(p=1), A.HorizontalFlip(),
                             A.RandomBrightnessContrast(brightness_limit=0.2),A.Resize(300,300)])
            image_aug = aug(image=image)['image']
            cv2.imwrite('train_300_5/' + str(i) + category + name, image_aug)

100%|██████████| 3/3 [00:00<00:00,  6.97it/s]


In [7]:
import re

glob.glob('/kaggle/working/*/*')
train_aug = pd.DataFrame({'path':glob.glob('/kaggle/working/*/*')})
train_aug['label'] = train_aug['path'].apply(lambda x: x.split('/')[-1].split('.')[0]).apply(lambda x: re.sub(r"[0-9]","",x))
train_aug

,path,label
0,/kaggle/working/train_300_5/2반점2.png,반점
1,/kaggle/working/train_300_5/2틈새과다1.png,틈새과다
2,/kaggle/working/train_300_5/2반점1.png,반점
3,/kaggle/working/train_300_5/5틈새과다0.png,틈새과다
4,/kaggle/working/train_300_5/2틈새과다0.png,틈새과다
...,...,...
643,"/kaggle/working/train_300_3/1창틀,문틀수정12.png","창틀,문틀수정"
644,/kaggle/working/train_300_3/2울음17.png,울음
645,"/kaggle/working/train_300_3/2창틀,문틀수정22.png","창틀,문틀수정"
646,"/kaggle/working/train_300_3/0창틀,문틀수정17.png","창틀,문틀수정"


In [8]:
train = pd.concat([train,train_aug], axis=0)  # 중복실행 X
train

,path,label
0,/kaggle/input/dacon-dobe2/train/가구수정/11.png,가구수정
1,/kaggle/input/dacon-dobe2/train/가구수정/4.png,가구수정
2,/kaggle/input/dacon-dobe2/train/가구수정/9.png,가구수정
3,/kaggle/input/dacon-dobe2/train/가구수정/1.png,가구수정
4,/kaggle/input/dacon-dobe2/train/가구수정/2.png,가구수정
...,...,...
643,"/kaggle/working/train_300_3/1창틀,문틀수정12.png","창틀,문틀수정"
644,/kaggle/working/train_300_3/2울음17.png,울음
645,"/kaggle/working/train_300_3/2창틀,문틀수정22.png","창틀,문틀수정"
646,"/kaggle/working/train_300_3/0창틀,문틀수정17.png","창틀,문틀수정"


## 04. Train & Prediction
- Cross Validation & Ensembling
- Test-Time Augmentation(TTA) : test data를 augmentation하여 추론 & prediction에 반영

In [9]:
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB1
from tensorflow.keras.callbacks import *


skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
#kf = KFold(n_splits=10, random_state=42,shuffle=True)
result = 0
result2 = 0
for train_idx,valid_idx in skf.split(train,train['label']):
    x_train = train.iloc[train_idx]
    x_valid = train.iloc[valid_idx]
        
    es = EarlyStopping(patience = 4,restore_best_weights=True)
    rl = ReduceLROnPlateau(patience = 3,verbose=1)
    
    idg = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)   ## Test-Time Augmentation (TTA) 수직,수평 뒤집기
    idg2 = ImageDataGenerator()
    
    train_generator = idg.flow_from_dataframe(x_train, x_col = 'path',
                                              y_col = 'label',batch_size=32,target_size=(300,300)) # aug Plus
    valid_generator = idg2.flow_from_dataframe(x_valid, x_col = 'path',
                                               y_col = 'label',batch_size=32,target_size=(300,300)) # aug No
    test_generator = idg.flow_from_dataframe(test, x_col = 'img_path',y_col = None, batch_size=32,
                                             class_mode = None, shuffle=False,target_size=(300,300))    
    test_generator2 = idg2.flow_from_dataframe(test, x_col = 'img_path',y_col = None, batch_size=32, 
                                               class_mode = None, shuffle=False,target_size=(300,300))    
    
    # 모델 선언
    model = Sequential()
    model.add(EfficientNetB4(include_top=False, pooling='avg'))
    model.add(Dense(19, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')
    model.fit(train_generator, validation_data=valid_generator, epochs=50, callbacks=[es,rl])    
    
    
    # 예측 : TTA 30%, 일반추론 70%씩 반영
    result += model.predict(test_generator)/10 # 30%
    result2 += model.predict(test_generator2)/10 # 70%
    
final_result = 0.3*result + 0.7*result2

Found 3694 validated image filenames belonging to 19 classes.
Found 411 validated image filenames belonging to 19 classes.
Found 792 validated image filenames.
Found 792 validated image filenames.
71686520/71686520 [==============================] - 3s 0us/step
Epoch 1/50


2023-05-21 04:49:08.719190: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb4/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


116/116 [==============================] - 198s 988ms/step - loss: 1.2481 - acc: 0.6267 - val_loss: 1.4533 - val_acc: 0.6569 - lr: 0.0010
Epoch 2/50
116/116 [==============================] - 107s 919ms/step - loss: 0.6495 - acc: 0.7945 - val_loss: 1.0603 - val_acc: 0.7178 - lr: 0.0010
Epoch 3/50
116/116 [==============================] - 107s 916ms/step - loss: 0.4632 - acc: 0.8525 - val_loss: 1.0113 - val_acc: 0.7518 - lr: 0.0010
Epoch 4/50
116/116 [==============================] - 106s 911ms/step - loss: 0.3938 - acc: 0.8720 - val_loss: 0.8696 - val_acc: 0.7689 - lr: 0.0010
Epoch 5/50
116/116 [==============================] - 106s 914ms/step - loss: 0.2819 - acc: 0.9071 - val_loss: 0.8827 - val_acc: 0.8078 - lr: 0.0010
Epoch 6/50
116/116 [==============================] - 106s 913ms/step - loss: 0.2797 - acc: 0.9112 - val_loss: 0.9641 - val_acc: 0.7835 - lr: 0.0010
Epoch 7/50
116/116 [==============================] - ETA: 0s - loss: 0.2464 - acc: 0.9285
Epoch 7: ReduceLROnPlateau

2023-05-21 05:18:38.319121: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb4/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


116/116 [==============================] - 184s 938ms/step - loss: 1.2535 - acc: 0.6302 - val_loss: 1.2319 - val_acc: 0.7080 - lr: 0.0010
Epoch 2/50
116/116 [==============================] - 110s 947ms/step - loss: 0.6635 - acc: 0.7943 - val_loss: 0.9774 - val_acc: 0.7324 - lr: 0.0010
Epoch 3/50
116/116 [==============================] - 106s 908ms/step - loss: 0.4630 - acc: 0.8506 - val_loss: 0.6176 - val_acc: 0.8370 - lr: 0.0010
Epoch 4/50
116/116 [==============================] - 106s 907ms/step - loss: 0.3588 - acc: 0.8882 - val_loss: 0.6954 - val_acc: 0.8200 - lr: 0.0010
Epoch 5/50
116/116 [==============================] - 106s 909ms/step - loss: 0.3111 - acc: 0.8958 - val_loss: 0.8861 - val_acc: 0.7567 - lr: 0.0010
Epoch 6/50
116/116 [==============================] - ETA: 0s - loss: 0.2577 - acc: 0.9185
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
116/116 [==============================] - 105s 905ms/step - loss: 0.2577 - acc: 0.9185 - val_loss

...

116/116 [==============================] - 187s 952ms/step - loss: 1.2490 - acc: 0.6436 - val_loss: 1.1308 - val_acc: 0.6732 - lr: 0.0010
Epoch 2/50
116/116 [==============================] - 107s 918ms/step - loss: 0.6668 - acc: 0.7892 - val_loss: 1.0656 - val_acc: 0.7220 - lr: 0.0010
Epoch 3/50
116/116 [==============================] - 107s 916ms/step - loss: 0.4526 - acc: 0.8560 - val_loss: 0.6308 - val_acc: 0.7878 - lr: 0.0010
Epoch 4/50
116/116 [==============================] - 106s 911ms/step - loss: 0.3754 - acc: 0.8793 - val_loss: 0.9477 - val_acc: 0.7610 - lr: 0.0010
Epoch 5/50
116/116 [==============================] - 107s 916ms/step - loss: 0.3049 - acc: 0.9031 - val_loss: 1.0142 - val_acc: 0.7439 - lr: 0.0010
Epoch 6/50
116/116 [==============================] - ETA: 0s - loss: 0.2871 - acc: 0.9077
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
116/116 [==============================] - 107s 917ms/step - loss: 0.2871 - acc: 0.9077 - val_loss

2023-05-21 09:05:42.750161: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_8/efficientnetb4/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


116/116 [==============================] - 191s 1s/step - loss: 1.2400 - acc: 0.6360 - val_loss: 1.3062 - val_acc: 0.6902 - lr: 0.0010
Epoch 2/50
116/116 [==============================] - 108s 930ms/step - loss: 0.6413 - acc: 0.7989 - val_loss: 0.7745 - val_acc: 0.7732 - lr: 0.0010
Epoch 3/50
116/116 [==============================] - 108s 929ms/step - loss: 0.4889 - acc: 0.8417 - val_loss: 0.7035 - val_acc: 0.7780 - lr: 0.0010
Epoch 4/50
116/116 [==============================] - 108s 930ms/step - loss: 0.3717 - acc: 0.8752 - val_loss: 0.6746 - val_acc: 0.8146 - lr: 0.0010
Epoch 5/50
116/116 [==============================] - 108s 928ms/step - loss: 0.2999 - acc: 0.9009 - val_loss: 0.7407 - val_acc: 0.7829 - lr: 0.0010
Epoch 6/50
116/116 [==============================] - 109s 935ms/step - loss: 0.2722 - acc: 0.9142 - val_loss: 0.6423 - val_acc: 0.8463 - lr: 0.0010
Epoch 7/50
116/116 [==============================] - 108s 929ms/step - loss: 0.2338 - acc: 0.9210 - val_loss: 0.7453 - 

2023-05-21 09:56:30.065957: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_9/efficientnetb4/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


116/116 [==============================] - 184s 948ms/step - loss: 1.1988 - acc: 0.6438 - val_loss: 1.0849 - val_acc: 0.6854 - lr: 0.0010
Epoch 2/50
116/116 [==============================] - 106s 914ms/step - loss: 0.6593 - acc: 0.7943 - val_loss: 0.9327 - val_acc: 0.7829 - lr: 0.0010
Epoch 3/50
116/116 [==============================] - 107s 916ms/step - loss: 0.4487 - acc: 0.8601 - val_loss: 1.2777 - val_acc: 0.7317 - lr: 0.0010
Epoch 4/50
116/116 [==============================] - 106s 914ms/step - loss: 0.3907 - acc: 0.8736 - val_loss: 0.9326 - val_acc: 0.7537 - lr: 0.0010
Epoch 5/50
116/116 [==============================] - 106s 913ms/step - loss: 0.3234 - acc: 0.8958 - val_loss: 0.7916 - val_acc: 0.7976 - lr: 0.0010
Epoch 6/50
116/116 [==============================] - 107s 915ms/step - loss: 0.2561 - acc: 0.9175 - val_loss: 1.1151 - val_acc: 0.7805 - lr: 0.0010
Epoch 7/50
116/116 [==============================] - 107s 918ms/step - loss: 0.2140 - acc: 0.9307 - val_loss: 0.9835

In [10]:
label_dict = {key:val for key,val in zip(train_generator.classes, x_train.label.tolist())}
prediction = np.argmax(final_result,axis=1)
prediction = np.vectorize(label_dict.get)(prediction)
prediction

array(['훼손', '오염', '훼손', '훼손', '오염', '훼손', '오염', '훼손', '오염', '오타공',
       '몰딩수정', '오타공', '오염', '오염', '훼손', '훼손', '훼손', '걸레받이수정', '오염', '오염',
       '가구수정', '훼손', '오염', '훼손', '가구수정', '훼손', '터짐', '곰팡이', '훼손', '훼손',
       '오염', '훼손', '훼손', '오염', '곰팡이', '훼손', '훼손', '훼손', '오타공', '훼손', '훼손',
       '훼손', '석고수정', '오염', '훼손', '석고수정', '오염', '훼손', '오염', '오염', '오염',
       '오염', '오염', '석고수정', '훼손', '훼손', '훼손', '곰팡이', '오염', '오염', '오타공',
       '몰딩수정', '훼손', '오염', '훼손', '꼬임', '오염', '오염', '훼손', '터짐', '오염', '오염',
       '오염', '면불량', '훼손', '터짐', '훼손', '오염', '훼손', '오염', '오염', '석고수정',
       '오염', '훼손', '걸레받이수정', '훼손', '훼손', '훼손', '오염', '오염', '오염', '훼손',
       '훼손', '오염', '훼손', '훼손', '훼손', '훼손', '훼손', '오염', '피스', '훼손', '훼손',
       '터짐', '훼손', '훼손', '가구수정', '훼손', '훼손', '훼손', '훼손', '훼손', '훼손', '훼손',
       '오염', '훼손', '훼손', '훼손', '석고수정', '훼손', '울음', '훼손', '훼손', '터짐', '훼손',
       '훼손', '훼손', '훼손', '훼손', '오염', '꼬임', '오염', '꼬임', '오염', '훼손', '오염',
       '석고수정', '훼손', '훼손', '오염', '오타공', '훼손', '오염', '오염',

In [11]:
sub = pd.read_csv('/kaggle/input/dacon-dobe2/sample_submission.csv')
sub['label'] = prediction
sub.to_csv('sub.csv', index=False)